## GenAI: Task-tuned embeddings for search

Solution enhances the search for specific business processes by accounting for language and intents of different user roles in the domain.

This sample notebook shows you how to execute GenAI: Task-tuned embeddings for search Algorithm from AWS Marketplace 

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

#### Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. Some hands-on experience using [Amazon SageMaker](https://aws.amazon.com/sagemaker/).
1. To use this algorithm successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to For Seller to update: Task-tuned embeddings for search. 

#### Contents:
1. [Subscribe to the algorithm](#1.-Subscribe-to-the-algorithm)
1. [Prepare dataset](#2.-Prepare-dataset)
	1. [Dataset format expected by the algorithm](#A.-Dataset-format-expected-by-the-algorithm)
	1. [Configure dataset](#B.-Configure-dataset)
	1. [Upload datasets to Amazon S3](#C.-Upload-datasets-to-Amazon-S3)
1. [Execute optimization model](#3.-Execute-optimization-model)
	1. [Set up environment](#3.1-Set-up-environment)
	1. [Execute model](#3.2-Execute-model)
    1. [Visualize Output](#3.3-Inspect-the-Output-in-S3)
1. [Clean-up](#4.-Clean-up)
	1. [Unsubscribe to the listing (optional)](#Unsubscribe-to-the-listing-(optional))


#### Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

### 1. Subscribe to the algorithm

To subscribe to the algorithm:
1. Open the algorithm listing page **GenAI: Task-tuned embeddings for search**
1. On the AWS Marketplace listing,  click on **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you agree with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn**. This is the algorithm ARN that you need to specify while training a custom ML model. Copy the ARN corresponding to your region and specify the same in the following cell.

In [55]:
algo_arn = "put your arn here"

### 2. Prepare dataset

In [21]:
import os
import json 
import uuid
import boto3
import pickle
import base64
import tarfile
from pprint import pprint
from PIL import Image

import numpy as np
import pandas as pd

import urllib.request
from urllib.parse import urlparse

import sagemaker as sage
from sagemaker import ModelPackage
from sagemaker import get_execution_role

#### A. Dataset format expected by the algorithm

The algorithm requires data in the format as described for best results:
* Input File name should be Input.zip
* The input data files must contain one train_doc folder with all raw docs and one config.json file.
* For detailed instructions, please refer sample notebook and algorithm input details

#### B. Configure dataset

In [22]:
training_dataset="input/Input.zip"

#### C. Upload datasets to Amazon S3

In [23]:
role = get_execution_role()

In [24]:
sagemaker_session = sage.Session()

bucket = sagemaker_session.default_bucket()
bucket

'sagemaker-us-east-2-786796469737'

In [25]:
# training input location
common_prefix = "task-tuned-embedding-for-search"
training_input_prefix = common_prefix + "/training-input-data"
TRAINING_WORKDIR = "input" #Input directory in Jupyter Server
training_input = sagemaker_session.upload_data(TRAINING_WORKDIR, key_prefix=training_input_prefix) #uploads data from jupyter server to S3
print("Training input uploaded to " + training_input)

Training input uploaded to s3://sagemaker-us-east-2-786796469737/task-tuned-embedding-for-search/training-input-data


## 3. Execute the training process

Now that dataset is available in an accessible Amazon S3 bucket, we are ready to execute a training pipeline to get clean sentiment class labels using clean-sentiment-classification-labels Algorithm. 

### 3.1 Set up environment

In [26]:
output_location = 's3://{}/task-tuned-embedding-for-search/{}'.format(bucket, 'Output')

### 3.2 Execute model

For information on creating an `Estimator` object, see [documentation](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html)

In [30]:
training_instance_type="ml.g4dn.2xlarge"

In [31]:
#Create an estimator object for running a training job
estimator = sage.algorithm.AlgorithmEstimator(
    algorithm_arn=algo_arn,
    base_job_name="task-tuned-embedding-for-search-algo",
    role=role,
    train_instance_count=1,
    train_instance_type=training_instance_type,
    input_mode="File",
    output_path=output_location,
    sagemaker_session=sagemaker_session,
    instance_count=1,
    instance_type=training_instance_type
)

#Run the training job.
estimator.fit({"training": training_input})

2024-07-10 07:44:58 Starting - Starting the training job...
2024-07-10 07:45:25 Starting - Preparing the instances for trainingProfilerReport-1720597497: InProgress
...
2024-07-10 07:45:51 Downloading - Downloading the training image............
2024-07-10 07:47:45 Training - Training image download completed. Training in progress.==========
== CUDA ==
CUDA Version 12.1.1
Container image Copyright (c) 2016-2023, NVIDIA CORPORATION & AFFILIATES. All rights reserved.
This container image and its contents are governed by the NVIDIA Deep Learning Container License.
By pulling and using the container, you accept the terms and conditions of this license:
https://developer.nvidia.com/ngc/nvidia-deep-learning-container-license
A copy of this license is made available in this container at /NGC-DL-CONTAINER-LICENSE for your convenience.
/opt/ml/input/data/training/Input.zip
file name /opt/ml/input/data/training/Input.zip
current working directory /opt/program
all files in opt ml ['input', 'model

See this [blog-post](https://aws.amazon.com/blogs/machine-learning/easily-monitor-and-visualize-metrics-while-training-models-on-amazon-sagemaker/) for more information how to visualize metrics during the process. You can also open the training job from [Amazon SageMaker console](https://console.aws.amazon.com/sagemaker/home?#/jobs/) and monitor the metrics/logs in **Monitor** section.

In [32]:
#output is available on following path
estimator.output_path

's3://sagemaker-us-east-2-786796469737/task-tuned-embedding-for-search/Output'

### 3.3 Inspect the Output in S3

In [33]:
parsed_url = urlparse(estimator.output_path)
bucket_name = parsed_url.netloc
file_key = parsed_url.path[1:]+'/'+estimator.latest_training_job.job_name+'/output/'+"model.tar.gz"

s3_client = sagemaker_session.boto_session.client('s3')
response = s3_client.get_object(Bucket = sagemaker_session.default_bucket(), Key = file_key)

In [34]:
bucketFolder = estimator.output_path.rsplit('/')[3] +'/Output/'+ estimator.latest_training_job.job_name+'/output/'+"model.tar.gz"

In [35]:
s3_conn = boto3.client("s3")
bucket_name=bucket
with open('output.tar.gz', 'wb') as f:
    s3_conn.download_fileobj(bucket_name, bucketFolder, f)
    print("Output file loaded from bucket")

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


Output file loaded from bucket


In [36]:
with tarfile.open('output.tar.gz') as file:
    file.extractall('./output')

### 4: Deploy model and verify results

Now you can deploy the model for performing real-time inference.

In [37]:
role = get_execution_role()

sagemaker_session = sage.Session()

bucket=sagemaker_session.default_bucket()
bucket

'sagemaker-us-east-2-786796469737'

In [38]:
model_name='marketplace-task-tune-embeddings'

content_type='application/zip'

real_time_inference_instance_type='ml.p3.2xlarge'
batch_transform_inference_instance_type='ml.p3.2xlarge'

#### A. Deploy trained model

In [39]:
from sagemaker.predictor import csv_serializer
predictor = estimator.deploy(1, real_time_inference_instance_type)

..........
-----------!

Once endpoint is created, you can perform real-time inference.

#### B. Create input payload

In [40]:
file_name = 'inference/Input.zip'

#### C. Perform real-time inference

In [41]:
output_file_name = 'inf_output.zip'

In [42]:
!aws sagemaker-runtime invoke-endpoint \
    --endpoint-name $predictor.endpoint_name \
    --body fileb://$file_name \
    --content-type $content_type \
    --region $sagemaker_session.boto_region_name \
    $output_file_name

{
    "ContentType": "application/zip",
    "InvokedProductionVariant": "AllTraffic"
}


#### D. Check output

In [48]:
import zipfile

zip_file_path = 'inf_output.zip'
csv_file_name = 'embeddings.csv'  

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    #if csv_file_name in zip_ref.namelist():
    with zip_ref.open(csv_file_name) as csv_file:
            df = pd.read_csv(csv_file)
            
        
df.head()

,text,text_embeds
0,What are the potential consequences for the Is...,[ 2.73016058e-02 -1.58562679e-02 -3.25590968e-...
1,"How can fluctuations in MAPCs, Trips, Adjusted...",[-1.75918860e-03 -4.99562882e-02 -3.45395990e-...
2,"How is ""Disability"" defined in the context of ...",[-3.71765718e-02 -1.44260339e-02 -2.85149738e-...
3,"According to the context information, what is ...",[-2.21168268e-02 -1.66630652e-02 5.34352520e-...
4,How can the 2025 Convertible Notes be redeemed...,[-1.70179419e-02 -5.43422848e-02 2.27773022e-...


### 4. Clean-up

In [49]:
#predictor=sage.predictor.Predictor(model_name, sagemaker_session,content_type)
predictor.delete_endpoint(delete_endpoint_config=True)

In [54]:
model.delete_model()

#### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the algorithm, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.